<a href="https://colab.research.google.com/github/Abhi-v-b/LP-5/blob/main/HPC/ASS4_Matrix_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-1xdus22e
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-1xdus22e
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=1f701232b08c638850e5bbcfa961680f7cb9b5272c3a93b04b33c8d54e9cd45a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ytg7tlm6/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%writefile matrix_mult.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include<omp.h>
#define N 4

float A[N][N], B[N][N], C[N][N]; // declaring matrices of NxN size
int main ()
{
  /* DECLARING VARIABLES */
  int i, j, m; // indices for matrix multiplication
  float t_1; // Execution time measures
  clock_t c_1, c_2;
  /* FILLING MATRICES WITH RANDOM NUMBERS */
  for(i=0;i<N;i++)
  {
    for(j=0;j<N;j++)
    {
	A[i][j]= (rand()%5);
	B[i][j]= (rand()%5);
    }
  }
  // Display input matrix A:
  printf("Matrix A:\n");
  for(i=0;i<N;i++)
  {
    for(j=0;j<N;j++)
    {
      printf("%f\t",A[i][j]);
    }
    printf("\n");
  }
  // Display input matrix B:
  printf("Matrix B:\n");
  for(i=0;i<N;i++)
  {
    for(j=0;j<N;j++)
    {
        printf("%f\t",B[i][j]);
    }
    printf("\n");
  }
  c_1=clock();  // time measure:
  /* MATRIX MULTIPLICATION */
  printf("Max number of threads: %i \n",omp_get_max_threads());
  #pragma omp parallel
  #pragma omp single
  {
     printf("Number of threads: %i \n",omp_get_num_threads());
  }
  #pragma omp parallel for private(m,j)
  // #pragma omp_set_num_threads(8)
  for(i=0;i<N;i++)
  {
    for(j=0;j<N;j++)
    {
       C[i][j]=0.; // set initial value of resulting matrix C = 0
       for(m=0;m<N;m++)
       {
	  C[i][j]=A[i][m]*B[m][j]+C[i][j];
       }
     }
  }
  // Display input matrix B:
  printf("Matrix C:\n");
  for(i=0;i<N;i++)
  {
    for(j=0;j<N;j++)
    {
       printf("%f\t",C[i][j]);
    }
    printf("\n");
  }
  /* TIME MEASURE + OUTPUT */
  c_2=clock();  // time measure:
  t_1 = (float)(c_2-c_1)/CLOCKS_PER_SEC; // in seconds; - time elapsed for job row-wise
  printf("Execution time: %f(in seconds) \n",t_1);
  /* TERMINATE PROGRAM */
  return 0;
}

Writing matrix_mult.c


In [ ]:
!g++ matrix_mult.c -fopenmp -o mat

In [ ]:
!./mat

Matrix A:
3.000000	2.000000	3.000000	1.000000	
4.000000	2.000000	0.000000	3.000000	
0.000000	2.000000	1.000000	2.000000	
2.000000	2.000000	2.000000	4.000000	
Matrix B:
1.000000	0.000000	0.000000	2.000000	
1.000000	2.000000	4.000000	1.000000	
1.000000	1.000000	3.000000	4.000000	
0.000000	3.000000	0.000000	2.000000	
Max number of threads: 2 
Number of threads: 2 
Matrix C:
8.000000	10.000000	17.000000	22.000000	
6.000000	13.000000	8.000000	16.000000	
3.000000	11.000000	11.000000	10.000000	
6.000000	18.000000	14.000000	22.000000	
Execution time: 0.000092(in seconds) 
